## Demo Redis
In this section, we learn how to manipulate data on Redis.

Required:
pip install redis

We cover:
* CRUD operations
* Table to Dataframe

Documentation: https://redis-py.readthedocs.io/en/stable/

In [1]:
!pip3 install redis

     |████████████████████████████████| 72 kB 540 kB/s  eta 0:00:01


### CRUD - Redis

In [1]:
import redis
from datetime import datetime

redis.__version__

'3.5.3'

In [2]:
print('connecting to Redis server...')
r = redis.Redis(host='localhost')

# r = redis.Redis(
#         host='localhost',
#         port=port,
#         password='password')

print('connected')

connecting to Redis server...
connected


In [3]:
#echo

ret = r.echo('ping')
ret

b'ping'

In [4]:
# insert and read data

r.set('key1', 'hello world')
val = r.get('key1')
val

b'hello world'

In [5]:
# insert multiple data type
import json

r.set('key2', 100)

data = {
    "name": "user 1",
    "email": "user1@email.com"
}
r.set('key3', json.dumps(data))

True

In [6]:
# get all keys - don't do this on production machine!

for key in r.scan_iter():
    print(key)
    val = r.get(key)
    print(val)

b'key2'
b'100'
b'key4'
b'30'
b'key3'
b'{"name": "user 1", "email": "user1@email.com"}'
b'key0'
b'10'
b'key1'
b'hello world'


In [8]:
# filtering key
for key in r.scan_iter('*1'):
    print(key)
    val = r.get(key)
    print(val)

b'key1'
b'hello world'


In [9]:
# update

r.set('key1', 'hello new world')
r.set('key2', 99)

r.get('key1'),r.get('key2')

(b'hello new world', b'99')

In [10]:
# delete key

r.delete('key1')

val = r.get('key1')
val

In [11]:
# delete all

r.delete('key2','key3')

# check key
for key in r.scan_iter():
    print(key)

b'key4'
b'key0'


## Redis - Pandas


sqlalchemy does not support for NoSQL like Redis

In [12]:
for i in range(5):
    r.set('key' + str(i),i*5 + 10)

print('data loaded')    

data loaded


In [13]:
# load Redis keys to pandas
import pandas as pd

keys = []
values = []
for key in r.scan_iter():
    keys.append(str(key, 'utf-8'))
    values.append(str(r.get(key), 'utf-8'))
    
df = pd.DataFrame(list(zip(keys, values)),
               columns =['Key', 'Value'])
df

,Key,Value
0,key2,20
1,key4,30
2,key3,25
3,key0,10
4,key1,15


In [14]:
# load dataframe
products = {'name': ['Product A1', 'Product A2', 'Product A3'],           
            'code': ['E01', 'E02', 'E03'],
            'price': [2.1, 3.6, 3.5],
            'quantity': [5, 7, 3],
            'created': ['2020-10-18 20:12:21', '2020-10-18 20:12:21', '2020-10-18 20:12:21']
           }

dataFrame   = pd.DataFrame(data=products) 
dataFrame

,name,code,price,quantity,created
0,Product A1,E01,2.1,5,2020-10-18 20:12:21
1,Product A2,E02,3.6,7,2020-10-18 20:12:21
2,Product A3,E03,3.5,3,2020-10-18 20:12:21


In [15]:
# insert dataframe to redis

r.set("exproducts", dataFrame.to_json(orient="records"))

True

In [16]:
# get dataframe from redis

df = pd.read_json(r.get("exproducts"))
df

,name,code,price,quantity,created
0,Product A1,E01,2.1,5,2020-10-18 20:12:21
1,Product A2,E02,3.6,7,2020-10-18 20:12:21
2,Product A3,E03,3.5,3,2020-10-18 20:12:21


## Drop Keys

In [17]:
# Delete keys
r.delete('exproducts')
print('done')

done
